In [1]:
import pandas as pd

import glob

from pathlib import Path
import csv

In [2]:
# Initial imports
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import model_from_json
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

%matplotlib inline

In [3]:
## Load pickle for exports and imports of data  
import pickle 
def load_obj(path):
    with open(path, 'rb') as f:
        return pickle.load(f)
    
def save_obj(obj, path ):
    with open(path, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [8]:
path = Path('../Resources/ftd_key_list_sorted.pkl')
ftd_key_list = load_obj(path)
path = Path('../Resources/etf_key_list_sorted.pkl')
etf_key_list = load_obj(path)
path = Path('../Resources/eqt_key_list_sorted.pkl')
eqt_key_list = load_obj(path)
path = Path('../Resources/finished_key_list.csv')
key_list_import_df = pd.read_csv(path,index_col=0)
#key_list_import_df

In [13]:
path = Path('../Technical_Indicators/NewCsvs/A_all_indicator_dfs.xlsx')
test_df = pd.read_excel(path,index_col=0)
test_df.sort_index()

,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,...,ShortVolumeNSDQ,ShortExemptVolumeNSDQ,TotalVolumeNSDQ,ShortVolumeNYSE,ShortExemptVolumeNYSE,TotalVolumeNYSE,bollinger_signal,dema_signal,adl_signal,rsi_signal
Date,,,,,,,,,,,,,,,,,,,,,
2017-01-04,46.930000,47.380001,46.820000,47.099998,45.304924,1821300,1821300,0.17,0.362,47.10000,...,225468,71032,500923,1276,0,6238,0.282486,0.521843,-0.250000,0.304976
2017-01-05,47.049999,47.070000,46.360001,46.540001,44.766277,1503700,1503700,-0.51,-1.084,46.65667,...,62804,0,255300,1829,0,14527,0.414851,0.520254,-0.550000,0.275281
2017-01-06,46.630001,48.070000,46.560001,47.990002,46.161011,2883400,2883400,1.36,2.917,47.54000,...,484643,0,1040485,27855,0,60048,0.226796,0.628276,-0.666667,0.453451
2017-01-09,48.009998,48.560001,47.910000,48.139999,46.305290,2575300,2575300,0.13,0.271,48.20333,...,284202,221,647104,10272,0,41819,0.314770,0.674345,-0.800000,0.617978
2017-01-10,48.340000,48.540001,47.820000,48.099998,46.266823,1560500,1560500,-0.24,-0.496,48.15333,...,103532,1000,371334,23092,0,41810,0.401937,0.696585,-0.850000,0.678973
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-25,157.900000,158.520000,156.750000,158.340000,158.340000,1097631,1097631,0.44,0.279,157.87000,...,83528,0,218273,6976,0,20350,0.880549,0.987292,0.633333,0.296148
2021-10-26,159.630000,159.880000,157.060100,158.190000,158.190000,1019631,1019631,-1.44,-0.902,158.37670,...,133949,336,260766,8141,33,13959,0.718321,0.988880,0.733333,0.170947
2021-10-27,158.300000,158.320000,154.830000,155.260000,155.260000,1485295,1485295,-3.04,-1.920,156.13667,...,88493,1432,227044,7298,118,18013,0.800646,0.981732,0.800000,0.085875


In [14]:
test_key_list = eqt_key_list[0:10]
len(test_key_list)

10

In [17]:
df = test_df.copy()

num_of_inputs = 20
num_of_outputs= 1

# Create the features set (X) and the target set (y)
X = df.drop(columns={'close','adjClose'}).values
y = df['close'].values

scaler = StandardScaler().fit(X)
X = scaler.transform(X)

# Define the model - deep neural network with two layers
nn2 = Sequential()

# First hidden layer
nn2.add(Dense(units=8, input_dim=num_of_inputs, activation="relu"))

# Second hidden layer
nn2.add(Dense(units=8, activation="relu"))

# Output layer
nn2.add(Dense(units=num_of_outputs, activation="linear"))

# Compile the model
nn2.compile(loss="mean_squared_error", optimizer="adam", metrics=["mse"])

# Fit the model
model_2 = nn2.fit(X, y, validation_split=0.3, epochs=800, verbose=0)

In [18]:
model_loss, model_accuracy = nn2.evaluate(X, y, verbose=0)
model_accuracy

1.0533783435821533

In [20]:
# Save model_2 as JSON
nn_json2 = nn2.to_json()

file_path = Path("../Resources/model_2.json")
with open(file_path, "w") as json_file:
    json_file.write(nn_json2)

# Save weights
file_path = "../Resources/model_2.h5"
nn2.save_weights("../Resources/model_2.h5")

In [21]:
# Load the saved model to make predictions

# load json and create model
file_path = Path("../Resources/model_2.json")
with open(file_path, "r") as json_file:
    model_json = json_file.read()
loaded_model2 = model_from_json(model_json)

# load weights into new model
file_path = "../Resources/model_2.h5"
loaded_model2.load_weights(file_path)

In [22]:
df2 = test_df.copy()
df2['predicted'] = loaded_model2.predict(X)
df_pred = df2[['close','predicted']]
df_pred

,close,predicted
Date,,
2017-01-04,47.099998,48.663036
2017-01-05,46.540001,45.451435
2017-01-06,47.990002,49.645905
2017-01-09,48.139999,49.029030
2017-01-10,48.099998,47.556061
...,...,...
2021-10-25,158.340000,158.182068
2021-10-26,158.190000,157.939743
2021-10-27,155.260000,155.034302
